In [96]:
import pandas as pd
from typing import List
import pickle
from sklearn.preprocessing import OneHotEncoder

In [80]:
# Cargar DataFrame desde un archivo pickle
with open('df_eda.pkl', 'rb') as file:
    df = pickle.load(file)



In [82]:

#Para esta variables se deveran separar por las comas y considerar cada variable como un elemento, para luego transformar a dummies
multivariables = ['CarreraAspiracional','ConfiguracionPreferida']

data = df
final = pd.DataFrame()

cols = multivariables 

    # Iterar sobre cada columna en la lista `cols`
for col in cols:
           
        # Paso 1: Dividir la columna en múltiples columnas
    split_cols = data[col].str.split(', ', expand=True)
        
        # Paso 2: Renombrar las columnas resultantes
    split_cols.columns = [f'{col}_var_{i+1}' for i in range(split_cols.shape[1])]
        
        # Paso 3: Crear variables dummies para cada columna dividida
    stack = split_cols.stack()
    dummies = pd.get_dummies(stack)
    dummies_group = dummies.groupby(level=0).sum()
        
        # Renombrar las columnas dummies
    dummies_group.columns = [f'{col}_{dummy_col}' for dummy_col in dummies_group.columns]
        
        # Concatenar al DataFrame final
    final = pd.concat([final, dummies_group], axis=1)
    
final

,CarreraAspiracional_Become a content Creator in some platform,CarreraAspiracional_Build and develop a Team,CarreraAspiracional_Business Operations in any organization,CarreraAspiracional_Design and Creative strategy in any company,CarreraAspiracional_Design and Develop amazing software,CarreraAspiracional_Look deeply into Data and generate insights,CarreraAspiracional_Manage and drive End-to-End Projects or Products,CarreraAspiracional_Teaching in any of the institutes/online or Offline,CarreraAspiracional_Work as a freelancer and do my thing my way,CarreraAspiracional_Work in a BPO setup for some well known client,ConfiguracionPreferida_Work alone,ConfiguracionPreferida_Work with 2 to 3 people in my team,ConfiguracionPreferida_Work with 5 to 6 people in my team,ConfiguracionPreferida_Work with 7 to 10 or more people in my team,ConfiguracionPreferida_Work with more than 10 people in my team
0,0,1,1,0,0,0,0,0,1,0,1,1,1,1,1
1,0,1,1,0,0,1,0,0,0,0,0,0,1,0,0
2,1,0,0,0,1,0,1,0,0,0,0,1,1,0,0
3,0,0,1,0,0,1,1,0,0,0,0,1,0,0,0
4,0,0,1,0,0,0,1,1,0,0,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
230,0,1,1,1,0,0,0,0,0,0,0,0,0,0,1
231,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0
232,0,0,1,1,0,1,0,0,0,0,0,0,0,0,1
233,0,0,0,0,1,0,1,1,0,0,0,0,1,0,0


In [113]:
# Diccionario para almacenar los encoders por columna
encoders = {}

# Preparar una lista para almacenar las columnas codificadas
encoded_list = []

# Iterar sobre cada columna en la lista `multivariables`
for col in multivariables:
    # Dividir la columna en múltiples columnas
    split_cols = df[col].str.split(', ', expand=True)
    
    # Combinar todas las subcolumnas en una sola columna para ajustar el encoder
    combined_data = split_cols.stack().unique().reshape(-1, 1)
    
    # Inicializar y ajustar el OneHotEncoder
    encoder = OneHotEncoder(sparse_output=True, handle_unknown='ignore')
    encoder.fit(combined_data)
    
    # Almacenar el encoder en el diccionario
    encoders[col] = encoder
    
    # Transformar las subcolumnas utilizando el encoder ya entrenado
    encoded_cols = encoder.transform(split_cols.stack().values.reshape(-1, 1))
    
    # Crear un DataFrame con las columnas codificadas
    encoded_df = pd.DataFrame.sparse.from_spmatrix(
        encoded_cols, 
        columns=[f'{col}_{name.replace("_x0_", "_")}' for name in encoder.get_feature_names_out()]
    )
    
    # Sumar las columnas codificadas para cada fila original
    summed_encoded_df = encoded_df.groupby(encoded_df.index // split_cols.shape[1]).sum()
    
    # Agregar al DataFrame final
    encoded_list.append(summed_encoded_df.reset_index(drop=True))

# Concatenar todos los DataFrames de las columnas codificadas
final = pd.concat(encoded_list, axis=1)

final.columns = final.columns.str.replace('_x0_', '_')

final


,CarreraAspiracional_Become a content Creator in some platform,CarreraAspiracional_Build and develop a Team,CarreraAspiracional_Business Operations in any organization,CarreraAspiracional_Design and Creative strategy in any company,CarreraAspiracional_Design and Develop amazing software,CarreraAspiracional_Look deeply into Data and generate insights,CarreraAspiracional_Manage and drive End-to-End Projects or Products,CarreraAspiracional_Teaching in any of the institutes/online or Offline,CarreraAspiracional_Work as a freelancer and do my thing my way,CarreraAspiracional_Work in a BPO setup for some well known client,ConfiguracionPreferida_Work alone,ConfiguracionPreferida_Work with 2 to 3 people in my team,ConfiguracionPreferida_Work with 5 to 6 people in my team,ConfiguracionPreferida_Work with 7 to 10 or more people in my team,ConfiguracionPreferida_Work with more than 10 people in my team
0,0,1.0,1.0,0,0,0,0,0,1.0,0,1.0,1.0,1.0,1.0,1.0
1,0,1.0,1.0,0,0,1.0,0,0,0,0,0,3.0,2.0,0,0
2,1.0,0,0,0,1.0,0,1.0,0,0,0,0,1.0,3.0,1.0,0
3,0,0,1.0,0,0,1.0,1.0,0,0,0,2.0,2.0,1.0,0,0
4,0,0,1.0,0,0,0,1.0,1.0,0,0,1.0,1.0,2.0,0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
230,0,1.0,1.0,1.0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN
231,0,0,1.0,0,0,1.0,0,0,1.0,0,NaN,NaN,NaN,NaN,NaN
232,0,0,1.0,1.0,0,1.0,0,0,0,0,NaN,NaN,NaN,NaN,NaN
233,0,0,0,0,1.0,0,1.0,1.0,0,0,NaN,NaN,NaN,NaN,NaN


In [97]:
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

In [98]:
encoder.fit(combined_data)

OneHotEncoder(handle_unknown='ignore', sparse_output=False)

In [99]:
# Preparar una lista para almacenar las columnas codificadas
encoded_list = []

# Iterar sobre cada columna en la lista `multivariables`
for col in multivariables:
    # Dividir la columna en múltiples columnas
    split_cols = df[col].str.split(', ', expand=True)
    
    # Transformar cada columna individualmente utilizando el encoder ya entrenado
    encoded_cols = encoder.transform(split_cols.stack().values.reshape(-1, 1)).reshape(split_cols.shape[0], -1)
    
    # Convertir la csr_matrix resultante a un DataFrame
    encoded_df = pd.DataFrame.sparse.from_spmatrix(encoded_cols, columns=encoder.get_feature_names_out())
    
    # Agregar al DataFrame final
    encoded_list.append(encoded_df)

# Concatenar todos los DataFrames de las columnas codificadas
final = pd.concat(encoded_list, axis=1)

print(final)


ValueError: X has 1 features, but OneHotEncoder is expecting 8 features as input.

In [90]:
#Para esta variables se deveran separar por las comas y considerar cada variable como un elemento, para luego transformar a dummies
multivariables = ['CarreraAspiracional','ConfiguracionPreferida']

data = df
final = pd.DataFrame()

cols = multivariables 

    # Iterar sobre cada columna en la lista `cols`
#for col in cols:
           
        # Paso 1: Dividir la columna en múltiples columnas
split_cols = df['CarreraAspiracional'].str.split(', ', expand=True)
    
#         # Paso 2: Renombrar las columnas resultantes
# split_cols.columns = [f'{cols}_var_{i+1}' for i in range(split_cols.shape[1])]
        
#         # Paso 3: Crear variables dummies para cada columna dividida
stack = split_cols.stack()
transformer = OneHotEncoder(sparse_output=False)

dummies = transformer.fit_transform(pd.DataFrame(stack))
# dummies_group = dummies.groupby(level=0).sum()
        
#         # Renombrar las columnas dummies
# dummies_group.columns = [f'{col}_{dummy_col}' for dummy_col in dummies_group.columns]
        
#         # Concatenar al DataFrame final
# final = pd.concat([final, dummies_group], axis=1)
    
# final
pd.DataFrame(dummies)

,0,1,2,3,4,5,6,7,8,9
0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
700,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
701,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
702,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
703,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [9]:
from sklearn.preprocessing import OneHotEncoder

In [33]:
df_multivar = df[multivariables]
df_multivar

,CarreraAspiracional,ConfiguracionPreferida
0,"Business Operations in any organization, Build...","Work alone, Work with 2 to 3 people in my team..."
1,"Business Operations in any organization, Build...",Work with 5 to 6 people in my team
2,Manage and drive End-to-End Projects or Produc...,"Work with 2 to 3 people in my team, Work with ..."
3,"Business Operations in any organization, Manag...",Work with 2 to 3 people in my team
4,Teaching in any of the institutes/online or Of...,"Work with 2 to 3 people in my team, Work with ..."
...,...,...
230,"Design and Creative strategy in any company, B...",Work with more than 10 people in my team
231,"Business Operations in any organization, Look ...",Work with 2 to 3 people in my team
232,"Design and Creative strategy in any company, B...",Work with more than 10 people in my team
233,Teaching in any of the institutes/online or Of...,Work with 5 to 6 people in my team


In [64]:
df
final = pd.DataFrame()
for i in multivariables:
    split_cols = df[i].str.split(', ', expand=True)
        
    # Paso 2: Renombrar las columnas resultantes
    split_cols.columns = [f'{i}_{j+1}' for j in range(split_cols.shape[1])]

    final = pd.concat([final, split_cols], axis=1)

final

,CarreraAspiracional_1,CarreraAspiracional_2,CarreraAspiracional_3,ConfiguracionPreferida_1,ConfiguracionPreferida_2,ConfiguracionPreferida_3,ConfiguracionPreferida_4,ConfiguracionPreferida_5
0,Business Operations in any organization,Build and develop a Team,Work as a freelancer and do my thing my way,Work alone,Work with 2 to 3 people in my team,Work with 5 to 6 people in my team,Work with 7 to 10 or more people in my team,Work with more than 10 people in my team
1,Business Operations in any organization,Build and develop a Team,Look deeply into Data and generate insights,Work with 5 to 6 people in my team,None,None,None,None
2,Manage and drive End-to-End Projects or Products,Design and Develop amazing software,Become a content Creator in some platform,Work with 2 to 3 people in my team,Work with 5 to 6 people in my team,None,None,None
3,Business Operations in any organization,Manage and drive End-to-End Projects or Products,Look deeply into Data and generate insights,Work with 2 to 3 people in my team,None,None,None,None
4,Teaching in any of the institutes/online or Of...,Business Operations in any organization,Manage and drive End-to-End Projects or Products,Work with 2 to 3 people in my team,Work with 5 to 6 people in my team,None,None,None
...,...,...,...,...,...,...,...,...
230,Design and Creative strategy in any company,Business Operations in any organization,Build and develop a Team,Work with more than 10 people in my team,None,None,None,None
231,Business Operations in any organization,Look deeply into Data and generate insights,Work as a freelancer and do my thing my way,Work with 2 to 3 people in my team,None,None,None,None
232,Design and Creative strategy in any company,Business Operations in any organization,Look deeply into Data and generate insights,Work with more than 10 people in my team,None,None,None,None
233,Teaching in any of the institutes/online or Of...,Manage and drive End-to-End Projects or Products,Design and Develop amazing software,Work with 5 to 6 people in my team,None,None,None,None


In [67]:
z = pd.DataFrame(final.stack())
z

0
0   CarreraAspiracional_1         Business Operations in any organization
    CarreraAspiracional_2                        Build and develop a Team
    CarreraAspiracional_3     Work as a freelancer and do my thing my way
    ConfiguracionPreferida_1                                   Work alone
    ConfiguracionPreferida_2           Work with 2 to 3 people in my team
...                                                                   ...
233 ConfiguracionPreferida_1           Work with 5 to 6 people in my team
234 CarreraAspiracional_1         Business Operations in any organization
    CarreraAspiracional_2             Design and Develop amazing software
    CarreraAspiracional_3       Become a content Creator in some platform
    ConfiguracionPreferida_1  Work with 7 to 10 or more people in my team

[1043 rows x 1 columns]

In [70]:
stack = pd.DataFrame(final.stack())


transformer = OneHotEncoder(sparse_output=True, handle_unknown='ignore')


dummies = transformer.fit_transform(final)

encoded_df = pd.DataFrame.sparse.from_spmatrix(dummies, columns=transformer.get_feature_names_out())

# dummies_group = encoded_df.groupby(level=0).sum()

# dummies_group
# # #     #     # Renombrar las columnas dummies
# dummies_group.columns = [f'{col}_{dummy_col}' for dummy_col in dummies_group.columns]
        
# dummies_group        
##############################



# for col in multivariables:
#     # Paso 1: Dividir la columna en múltiples columnas
#     split_cols = df[col].str.split(', ', expand=True)
    
#     # Crear un OneHotEncoder
#     encoder = OneHotEncoder(sparse_output=True, handle_unknown='ignore')
    
#     # Ajustar y transformar las columnas divididas
#     encoded_cols = encoder.fit_transform(split_cols)
    
#     # Convertir la csr_matrix resultante a un DataFrame
#     encoded_df = pd.DataFrame.sparse.from_spmatrix(encoded_cols, columns=encoder.get_feature_names_out())
    
#     # Agrupar las variables dummies por nivel 0 (el índice original)
#     grouped_dummies = encoded_df.groupby(encoded_df.index).sum()
    
#     # Concatenar al DataFrame final
#     final = pd.concat([final, grouped_dummies], axis=1)

# print(final)


In [72]:
stack

0
0   CarreraAspiracional_1         Business Operations in any organization
    CarreraAspiracional_2                        Build and develop a Team
    CarreraAspiracional_3     Work as a freelancer and do my thing my way
    ConfiguracionPreferida_1                                   Work alone
    ConfiguracionPreferida_2           Work with 2 to 3 people in my team
...                                                                   ...
233 ConfiguracionPreferida_1           Work with 5 to 6 people in my team
234 CarreraAspiracional_1         Business Operations in any organization
    CarreraAspiracional_2             Design and Develop amazing software
    CarreraAspiracional_3       Become a content Creator in some platform
    ConfiguracionPreferida_1  Work with 7 to 10 or more people in my team

[1043 rows x 1 columns]

In [71]:
encoded_df

,CarreraAspiracional_1_Build and develop a Team,CarreraAspiracional_1_Business Operations in any organization,CarreraAspiracional_1_Design and Creative strategy in any company,CarreraAspiracional_1_Design and Develop amazing software,CarreraAspiracional_1_Look deeply into Data and generate insights,CarreraAspiracional_1_Manage and drive End-to-End Projects or Products,CarreraAspiracional_1_Teaching in any of the institutes/online or Offline,CarreraAspiracional_2_Build and develop a Team,CarreraAspiracional_2_Business Operations in any organization,CarreraAspiracional_2_Design and Develop amazing software,...,ConfiguracionPreferida_2_None,ConfiguracionPreferida_3_Work with 5 to 6 people in my team,ConfiguracionPreferida_3_Work with 7 to 10 or more people in my team,ConfiguracionPreferida_3_Work with more than 10 people in my team,ConfiguracionPreferida_3_None,ConfiguracionPreferida_4_Work with 7 to 10 or more people in my team,ConfiguracionPreferida_4_Work with more than 10 people in my team,ConfiguracionPreferida_4_None,ConfiguracionPreferida_5_Work with more than 10 people in my team,ConfiguracionPreferida_5_None
0,0,1.0,0,0,0,0,0,1.0,0,0,...,0,1.0,0,0,0,1.0,0,0,1.0,0
1,0,1.0,0,0,0,0,0,1.0,0,0,...,1.0,0,0,0,1.0,0,0,1.0,0,1.0
2,0,0,0,0,0,1.0,0,0,0,1.0,...,0,0,0,0,1.0,0,0,1.0,0,1.0
3,0,1.0,0,0,0,0,0,0,0,0,...,1.0,0,0,0,1.0,0,0,1.0,0,1.0
4,0,0,0,0,0,0,1.0,0,1.0,0,...,0,0,0,0,1.0,0,0,1.0,0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
230,0,0,1.0,0,0,0,0,0,1.0,0,...,1.0,0,0,0,1.0,0,0,1.0,0,1.0
231,0,1.0,0,0,0,0,0,0,0,0,...,1.0,0,0,0,1.0,0,0,1.0,0,1.0
232,0,0,1.0,0,0,0,0,0,1.0,0,...,1.0,0,0,0,1.0,0,0,1.0,0,1.0
233,0,0,0,0,0,0,1.0,0,0,0,...,1.0,0,0,0,1.0,0,0,1.0,0,1.0


In [55]:
final

,CarreraAspiracional_var_1,CarreraAspiracional_var_2,CarreraAspiracional_var_3,ConfiguracionPreferida_var_1,ConfiguracionPreferida_var_2,ConfiguracionPreferida_var_3,ConfiguracionPreferida_var_4,ConfiguracionPreferida_var_5
0,Business Operations in any organization,Build and develop a Team,Work as a freelancer and do my thing my way,Work alone,Work with 2 to 3 people in my team,Work with 5 to 6 people in my team,Work with 7 to 10 or more people in my team,Work with more than 10 people in my team
1,Business Operations in any organization,Build and develop a Team,Look deeply into Data and generate insights,Work with 5 to 6 people in my team,None,None,None,None
2,Manage and drive End-to-End Projects or Products,Design and Develop amazing software,Become a content Creator in some platform,Work with 2 to 3 people in my team,Work with 5 to 6 people in my team,None,None,None
3,Business Operations in any organization,Manage and drive End-to-End Projects or Products,Look deeply into Data and generate insights,Work with 2 to 3 people in my team,None,None,None,None
4,Teaching in any of the institutes/online or Of...,Business Operations in any organization,Manage and drive End-to-End Projects or Products,Work with 2 to 3 people in my team,Work with 5 to 6 people in my team,None,None,None
...,...,...,...,...,...,...,...,...
230,Design and Creative strategy in any company,Business Operations in any organization,Build and develop a Team,Work with more than 10 people in my team,None,None,None,None
231,Business Operations in any organization,Look deeply into Data and generate insights,Work as a freelancer and do my thing my way,Work with 2 to 3 people in my team,None,None,None,None
232,Design and Creative strategy in any company,Business Operations in any organization,Look deeply into Data and generate insights,Work with more than 10 people in my team,None,None,None,None
233,Teaching in any of the institutes/online or Of...,Manage and drive End-to-End Projects or Products,Design and Develop amazing software,Work with 5 to 6 people in my team,None,None,None,None


In [19]:
split_cols = df['CarreraAspiracional'].str.split(', ', expand=True)
        
    #     # Paso 2: Renombrar las columnas resultantes
split_cols.columns = [f'{'CarreraAspiracional'}_var_{i+1}' for i in range(split_cols.shape[1])]
        
    #     # Paso 3: Crear variables dummies para cada columna dividida
stack = pd.DataFrame(split_cols.stack(), columns=['CarreraAspiracional'])

transformer = OneHotEncoder()

dummies = transformer.fit_transform(stack)
dummies_group = dummies.groupby(level=0).sum()
        
    #     # Renombrar las columnas dummies
dummies_group.columns = [f'{col}_{dummy_col}' for dummy_col in dummies_group.columns]
        
    #     # Concatenar al DataFrame final
final = pd.concat([final, dummies_group], axis=1)

final

AttributeError: 'csr_matrix' object has no attribute 'groupby'

In [28]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder

# Ejemplo de DataFrame
df = pd.DataFrame({
    'CarreraAspiracional': ['Ingeniería, Medicina', 'Derecho, Ingeniería', 'Medicina']
})

# Paso 1: Dividir la columna 'CarreraAspiracional' en varias columnas
split_cols = df['CarreraAspiracional'].str.split(', ', expand=True)

# Paso 2: Renombrar las columnas resultantes
split_cols.columns = [f'CarreraAspiracional_var_{i+1}' for i in range(split_cols.shape[1])]

# Paso 3: Crear variables dummies para cada columna dividida
stack = pd.DataFrame(split_cols.stack(), columns=['CarreraAspiracional'])

# Usar OneHotEncoder
transformer = OneHotEncoder(sparse_output=False)  # Usar sparse=False para obtener un DataFrame en lugar de una matriz dispersa


# Ajustar y transformar
dummies = transformer.fit_transform(stack)

# Convertir a DataFrame para obtener los nombres de las columnas
dummies_df = pd.DataFrame(dummies, columns=transformer.get_feature_names_out(['CarreraAspiracional']))

# Agregar el índice del stack original para poder hacer un groupby
dummies_df.index = stack.index

# Agrupar por el índice original del DataFrame
dummies_group = dummies_df.groupby(level=0).sum()

# Concatenar al DataFrame original
final_df = pd.concat([df, dummies_group], axis=1)

final_df


,CarreraAspiracional,CarreraAspiracional_Derecho,CarreraAspiracional_Ingeniería,CarreraAspiracional_Medicina
0,"Ingeniería, Medicina",0.0,1.0,1.0
1,"Derecho, Ingeniería",1.0,1.0,0.0
2,Medicina,0.0,0.0,1.0
